In [19]:
import tensorflow as tf
import keras
#from keras.backend.tensorflow_backend import set_session
#from tensorflow.python.keras import backend as K


import numpy as np
from sklearn.model_selection import train_test_split
import os, glob
import matplotlib.pyplot as plt
import cv2
from keras.applications.mobilenet_v2 import MobileNetV2

from DataGenerator import DataGenerator


config = tf.compat.v1.ConfigProto()
#config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = False
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
base_path = '/mnt2/iris_processed/'

json_list = glob.glob(os.path.join(base_path, '*/data/*.json'))

json_list[:10]

In [ ]:
# np.savez_compressed(os.path.join(base_path, 'json_list.npz'), data=json_list)

base_path = '/mnt2/iris_processed/'
json_list = np.load(os.path.join(base_path, 'json_list.npz'))['data']

In [ ]:
print(len(json_list))

In [ ]:
json_list_train, json_list_val = train_test_split(json_list, test_size=0.1, random_state=2020)

print(len(json_list_train), len(json_list_val))